# Notebook to Pre-Calculate Median Incomes for State - Indiana

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import geopandas as gpd
import pandas as pd

## Download Shp Files for Surrounding States - Indiana

#### For ease, you can use the Basic Indiana Food Desert Analysis Notebook to download Tract Shape Files for Each State

In [15]:
# read the shape files of states

Illinois = gpd.read_file("Illinois/tl_2019_17_tract.shp")
Indiana = gpd.read_file("Indiana/tl_2019_18_tract.shp")
Kentucky = gpd.read_file("Kentucky/tl_2019_21_tract.shp")
Michigan = gpd.read_file("Michigan/tl_2019_26_tract.shp")
Ohio = gpd.read_file("Ohio/tl_2019_39_tract.shp")
Wisconsin = gpd.read_file("Wisconsin/tl_2019_55_tract.shp")

## Download Excel File for Income (2019 or Latest)

#### Go to census.data.gov , "Advanced Filters" > "Income and Poverty" > 2019 > "All Census Tracts within State of Choice (Indiana. here)"

In [11]:
# read the income data

Income = pd.read_excel("Income.xlsx")

In [13]:
Income

,OBJECTID,GEOID,NAME,S1901_C01_001E,S1901_C01_001M,S1901_C01_002E,S1901_C01_002M,S1901_C01_003E,S1901_C01_003M,S1901_C01_004E,...,S1901_C03_015M,S1901_C04_015E,S1901_C04_015M,S1901_C02_016E,S1901_C02_016M,S1901_C03_016E,S1901_C03_016M,S1901_C04_016E,S1901_C04_016M,Field131
0,2,17001000100,"Census Tract 1, Adams County, Illinois",2133,134,5.3,3.2,3.4,2.8,12.5,...,(X),(X),(X),(X),(X),(X),(X),34.4,(X),NaN
1,3,17001000201,"Census Tract 2.01, Adams County, Illinois",890,65,3.3,2.4,8.5,4.7,15.3,...,(X),(X),(X),(X),(X),(X),(X),22.2,(X),NaN
2,4,17001000202,"Census Tract 2.02, Adams County, Illinois",933,91,6.9,5.0,0.0,2.9,10.9,...,(X),(X),(X),(X),(X),(X),(X),27.9,(X),NaN
3,5,17001000400,"Census Tract 4, Adams County, Illinois",1273,128,13.1,5.0,8.6,4.5,15.4,...,(X),(X),(X),(X),(X),(X),(X),27.1,(X),NaN
4,6,17001000500,"Census Tract 5, Adams County, Illinois",850,75,6.9,4.5,5.3,4.2,20.6,...,(X),(X),(X),(X),(X),(X),(X),36.4,(X),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12918,12920,55141011300,"Census Tract 113, Wood County, Wisconsin",1993,139,2.0,2.0,8.2,5.2,8.8,...,(X),(X),(X),(X),(X),(X),(X),35.9,(X),NaN
12919,12921,55141011400,"Census Tract 114, Wood County, Wisconsin",2452,143,5.9,3.6,5.3,2.8,12.3,...,(X),(X),(X),(X),(X),(X),(X),32.1,(X),NaN
12920,12922,55141011500,"Census Tract 115, Wood County, Wisconsin",2342,119,2.3,2.3,0.8,1.2,7.5,...,(X),(X),(X),(X),(X),(X),(X),35.1,(X),NaN
12921,12923,55141011600,"Census Tract 116, Wood County, Wisconsin",2235,126,7.6,4.9,3.4,3.6,5.0,...,(X),(X),(X),(X),(X),(X),(X),16.4,(X),NaN


In [33]:
income_subset = Income.iloc[:, [1, 3, 25, 27, 35, 101, 107]].rename(columns={
    Income.columns[3]: 'tothouseholds',
    Income.columns[25]: 'medhouse',
    Income.columns[27]: 'meanhouse',
    Income.columns[35]: 'totfam',
    Income.columns[101]: 'medfam',
    Income.columns[107]: 'meanfam',
    Income.columns[1]: 'GEOID'
})

In [35]:
income_subset

,GEOID,tothouseholds,medhouse,meanhouse,totfam,medfam,meanfam
0,17001000100,2133,44613,71752,1116,58000,83711
1,17001000201,890,44878,53336,579,54509,61522
2,17001000202,933,46964,60429,593,61887,74103
3,17001000400,1273,33750,44740,712,39444,51721
4,17001000500,850,38526,40834,422,47727,49399
...,...,...,...,...,...,...,...
12918,55141011300,1993,49310,60471,1182,62872,72620
12919,55141011400,2452,44245,58853,1318,69310,78418
12920,55141011500,2342,77098,86480,1718,87807,96133
12921,55141011600,2235,58537,71340,1489,69289,83089


## Combining all Shape Files with Income Data

In [19]:
# Combine all state shapefiles into one GeoDataFrame
states_list = [Illinois, Indiana, Kentucky, Michigan, Ohio, Wisconsin]
combined_states = gpd.GeoDataFrame(pd.concat(states_list, ignore_index=True))

In [37]:
# Ensure GEOID columns are the same type (string) in both DataFrames
combined_states['GEOID'] = combined_states['GEOID'].astype(str)
income_subset['GEOID'] = income_subset['GEOID'].astype(str)

# Merge the combined shapefiles with income data
merged_data = combined_states.merge(income_subset, on='GEOID', how='left')

In [39]:
merged_data

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tothouseholds,medhouse,meanhouse,totfam,medfam,meanfam
0,17,091,011700,17091011700,117,Census Tract 117,G5020,S,2370100,102060,+41.1294653,-087.8735796,"POLYGON ((-87.88768 41.13594, -87.88764 41.136...",1379,22823,34995,814,20366,29993
1,17,091,011800,17091011800,118,Census Tract 118,G5020,S,1791332,55670,+41.1403452,-087.8760059,"POLYGON ((-87.8941 41.14388, -87.894 41.14387,...",1018,53235,58925,598,67941,70074
2,17,119,400951,17119400951,4009.51,Census Tract 4009.51,G5020,S,5169973,169066,+38.7277628,-090.1002620,"POLYGON ((-90.11192 38.70281, -90.11128 38.703...",2181,63162,72427,1303,85221,97956
3,17,119,400952,17119400952,4009.52,Census Tract 4009.52,G5020,S,5751299,305906,+38.7301416,-090.0827783,"POLYGON ((-90.09442 38.72031, -90.0936 38.7203...",1107,59271,65165,822,65484,73496
4,17,135,957500,17135957500,9575,Census Tract 9575,G5020,S,450037682,512225,+39.3934535,-089.5975234,"POLYGON ((-89.70369 39.34803, -89.69928 39.348...",1175,55083,68161,828,66389,79048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12918,55,025,012001,55025012001,120.01,Census Tract 120.01,G5020,S,23279636,19365,+43.0916914,-089.1973312,"POLYGON ((-89.24703 43.10718, -89.24703 43.107...",2926,92615,106453,2502,100284,114897
12919,55,079,002500,55079002500,25,Census Tract 25,G5020,S,555003,0,+43.1079640,-087.9621803,"POLYGON ((-87.96652 43.10481, -87.96651 43.105...",764,32162,45341,491,35586,48292
12920,55,025,012100,55025012100,121,Census Tract 121,G5020,S,150817337,2909705,+42.8834878,-089.1179994,"POLYGON ((-89.25035 42.88905, -89.25011 42.889...",1114,71400,85544,751,83438,93426
12921,55,025,010800,55025010800,108,Census Tract 108,G5020,S,16959326,9971,+43.0037370,-089.5400283,"POLYGON ((-89.57819 43.00247, -89.57819 43.002...",4542,103500,111435,3022,125035,134036


In [41]:
# Save the merged data as a new shapefile
merged_data.to_file("income_with_shapes.shp")

## The saved shape file was processed in GIS application (for now) to Keep Metro Areas

#### PySAL will be Used soon to automate this in Python

In [43]:
ShapeIncome = gpd.read_file("Metro_Income/Income_PairwiseIntersect.shp")

In [45]:
ShapeIncome

,FID_income,STATEFP,COUNTYFP,TRACTCE,GEOID,ALAND,AWATER,INTPTLAT,INTPTLON,tothouseho,...,FID_Metro_,CSAFP,CBSAFP,AFFGEOID,GEOID_1,NAME,LSAD,ALAND_1,AWATER_1,geometry
0,3300,18,141,011601,18141011601,15001576.0,417109.0,+41.6936056,-086.0916354,2733.0,...,8,515,21140,310M400US21140,21140,"Elkhart-Goshen, IN",M1,1.199604e+09,12434164.0,"MULTIPOLYGON (((-86.06139 41.68199, -86.06141 ..."
1,3308,18,141,011405,18141011405,9814048.0,0.0,+41.7367499,-086.0729349,1212.0,...,8,515,21140,310M400US21140,21140,"Elkhart-Goshen, IN",M1,1.199604e+09,12434164.0,"MULTIPOLYGON (((-86.062 41.70993, -86.06216 41..."
2,3525,18,039,002600,18039002600,1942940.0,0.0,+41.6739499,-085.9697466,1466.0,...,8,515,21140,310M400US21140,21140,"Elkhart-Goshen, IN",M1,1.199604e+09,12434164.0,"POLYGON ((-85.98489 41.67405, -85.98487 41.674..."
3,3526,18,039,002700,18039002700,1040909.0,54144.0,+41.6827398,-085.9788811,768.0,...,8,515,21140,310M400US21140,21140,"Elkhart-Goshen, IN",M1,1.199604e+09,12434164.0,"POLYGON ((-85.98494 41.68112, -85.98474 41.681..."
4,3527,18,039,002000,18039002000,18161685.0,19015.0,+41.6344664,-085.9054855,2721.0,...,8,515,21140,310M400US21140,21140,"Elkhart-Goshen, IN",M1,1.199604e+09,12434164.0,"POLYGON ((-85.92619 41.63069, -85.92618 41.630..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4295,5185,21,059,001800,21059001800,233401991.0,4511768.0,+37.6928847,-087.2646189,2366.0,...,13,None,21780,310M400US21780,21780,"Evansville, IN-KY",M1,3.792648e+09,125972863.0,"MULTIPOLYGON (((-87.40855 37.68356, -87.40648 ..."
4296,5233,21,059,001300,21059001300,133250730.0,14238965.0,+37.8176665,-087.2382484,2445.0,...,13,None,21780,310M400US21780,21780,"Evansville, IN-KY",M1,3.792648e+09,125972863.0,"MULTIPOLYGON (((-87.29681 37.75638, -87.29481 ..."
4297,5689,21,101,020500,21101020500,2008162.0,711693.0,+37.8265662,-087.5984488,937.0,...,13,None,21780,310M400US21780,21780,"Evansville, IN-KY",M1,3.792648e+09,125972863.0,"POLYGON ((-87.61444 37.8325, -87.61351 37.8329..."
4298,5723,21,225,950201,21225950201,207628817.0,3325925.0,+37.6798023,-087.8288943,1658.0,...,13,None,21780,310M400US21780,21780,"Evansville, IN-KY",M1,3.792648e+09,125972863.0,"MULTIPOLYGON (((-87.73455 37.63838, -87.73563 ..."


In [49]:
# Convert 'medfam' to numeric, coercing errors to NaN
ShapeIncome['medfam'] = pd.to_numeric(ShapeIncome['medfam'], errors='coerce')

# Check for any non-numeric values that became NaN
if ShapeIncome['medfam'].isna().any():
    print(f"Warning: {ShapeIncome['medfam'].isna().sum()} rows had non-numeric values in 'medfam' and were converted to NaN")

## Group the Metro Areas by Name and Find Median Family Income

In [51]:
median_by_name = ShapeIncome.groupby('NAME')['medfam'].median().reset_index()

In [53]:
median_by_name.columns = ['Name', 'Median_Income']

In [55]:
median_by_name

,Name,Median_Income
0,"Bloomington, IN",66827.0
1,"Chicago-Naperville-Elgin, IL-IN-WI",78531.5
2,"Cincinnati, OH-KY-IN",73438.0
3,"Columbus, IN",70625.0
4,"Elkhart-Goshen, IN",67910.0
5,"Evansville, IN-KY",62131.0
6,"Fort Wayne, IN",64881.0
7,"Indianapolis-Carmel-Anderson, IN",67000.0
8,"Kokomo, IN",63516.0
9,"Lafayette-West Lafayette, IN",64500.0


#### Save the File as CSV to Read it in LILA Plotting

In [57]:
# Save to CSV file
median_by_name.to_csv('Metro_Indiana_Income.csv', index=False)